In [1]:
import fastf1
import matplotlib.pyplot as plt

In [ ]:
session = fastf1.get_session(2024, "Belgium", 'R')

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule


In [10]:
session.load()

core           INFO 	Loading data for Belgian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
logger      WARNING 	Failed to add first lap time from Ergast!
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
c

In [11]:
lap = session.laps.pick_fastest()
tel = lap.get_telemetry()

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


In [ ]:
dflaps = session.laps

dflaps[dflaps["DriverNumber"] == "11"][["LapStartTime","LapStartDate","Time","DriverNumber","LapNumber"]]

In [ ]:
dflaps[dflaps["DriverNumber"] == "44"][["LapStartTime","LapStartDate","Time","DriverNumber","LapNumber"]]

In [12]:
from plotly import express as px
px.scatter(x = tel.X, y = tel.Y)

In [13]:
import livef1
from livef1.utils.constants import interpolation_map

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

session = livef1.get_session(season=2024, meeting_identifier="Spa", session_identifier="Race")

res = session.get_data(dataName="Position")
df_pos = pd.DataFrame(res.value)
df_pos["Utc"] = pd.to_datetime(df_pos["Utc"])
df_pos["timestamp"] = pd.to_datetime(df_pos["timestamp"])
df_pos["min"] = df_pos["Utc"].dt.minute
df_pos["hour"] = df_pos["Utc"].dt.hour
df_pos["sec"] = df_pos["Utc"].dt.second

res = session.get_data(dataName="Car_Data")
df_car = pd.DataFrame(res.value)
df_car["Utc"] = pd.to_datetime(df_car["Utc"])
df_car["timestamp"] = pd.to_datetime(df_car["timestamp"])
df_car["min"] = df_car["Utc"].dt.minute
df_car["hour"] = df_car["Utc"].dt.hour
df_car["sec"] = df_car["Utc"].dt.second

df = df_car.set_index("Utc").join(df_pos.set_index("Utc"), rsuffix="_pos", how="outer")

df_44 = df[(df["DriverNo"] == "44") | (df["DriverNo_pos"] == "44")]
df_44["X"] = df_44.reset_index()["X"].astype(float).interpolate(method = "quadratic", order = 2).values
df_44["Y"] = df_44.reset_index()["Y"].astype(float).interpolate(method = "quadratic", order = 2).values

df_temp = df_44.iloc[22000:23900]

2025-01-17 17:06:12,972 : livef1 : INFO :: Selected meeting/session is:
Meeting Offname : FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
Meeting Name : Belgian Grand Prix
Meeting Circuit Shortname : Spa-Francorchamps
2025-01-17 17:06:12,974 : livef1 : INFO :: The meeting was received successfully.
2025-01-17 17:06:12,981 : livef1 : INFO :: Selected meeting/session is:
Session Name : Race
2025-01-17 17:06:12,982 : livef1 : INFO :: The session was received successfully.
2025-01-17 17:06:13,574 : livef1 : INFO :: Getting requested data.
Selected session : 2024 Belgian Grand Prix Race
Topic : Position.z
2025-01-17 17:06:17,884 : livef1 : INFO :: Data is successfully received.
2025-01-17 17:06:19,570 : livef1 : INFO :: Data is successfully parsed.
C:\Users\EU10698\AppData\Local\Temp\ipykernel_20100\964930564.py:14: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.


In [14]:
df_11 = df[(df["DriverNo"] == "11") | (df["DriverNo_pos"] == "11")].reset_index()
df_11["Utc"] = pd.to_datetime(df_11["Utc"].values).tz_localize("UTC")

df_11["X"] = df_11.reset_index()["X"].astype(float).interpolate(method = "quadratic", order = 2).values
df_11["Y"] = df_11.reset_index()["Y"].astype(float).interpolate(method = "quadratic", order = 2).values

start = pd.Timestamp(lap.LapStartDate).tz_localize('UTC')
end = pd.Timestamp(lap.LapStartDate + lap.LapTime).tz_localize('UTC')
df_temp = df_11[(df_11["Utc"] >= start) & (df_11["Utc"] <= end)]
df_temp

import plotly.graph_objects as go
from plotly.subplots import make_subplots

df1 = df_temp[df_temp["SessionKey"].isnull()]
df1 = df_temp.copy()
trace1 = go.Scatter(
    x=df1["X"],
    y=df1["Y"],
    name="LiveF1",
    mode="markers"
)
trace2 = go.Scatter(
    x=tel["X"],
    y=tel["Y"],
    name="FastF1",
    mode="markers"
)

fig = make_subplots()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig

In [ ]:
df_11 = df[(df["DriverNo"] == "11") | (df["DriverNo_pos"] == "11")].reset_index()
df_11["Utc"] = pd.to_datetime(df_11["Utc"].values).tz_localize("UTC")

start = pd.Timestamp(lap.LapStartDate).tz_localize('UTC')
end = pd.Timestamp(lap.LapStartDate + lap.LapTime).tz_localize('UTC')
df_temp = df_11[(df_11["Utc"] >= start) & (df_11["Utc"] <= end)]
df_temp

import plotly.graph_objects as go
from plotly.subplots import make_subplots

df1 = df_temp[~df_temp["SessionKey"].isnull()]
trace1 = go.Scatter(
    x=df1["Utc"],
    y=df1["speed"],
    name="LiveF1",
    mode="markers"
)
trace2 = go.Scatter(
    x=tel["Date"],
    y=tel["Speed"],
    name="FastF1",
    mode="markers"
)

fig = make_subplots()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig

In [ ]:
(tel["Date"] - tel["Date"].shift(1)).dt.total_seconds().mean()

In [ ]:
(df1.reset_index()["Utc"] - df1.reset_index()["Utc"].shift(1)).dt.total_seconds().mean()

In [ ]:
df1

In [ ]:
LapSeries